In [ ]:
import importlib

import numpy as np

package = "tensorflow"
try:
    importlib.import_module(package)
except ImportError:
    !pip install - U {package}
    importlib.import_module(package)
package = "transformers"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "pandas"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "sklearn"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "keras"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "torch"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "transformers"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "langdetect"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

import torch
from sklearn.model_selection import train_test_split
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import classification_report
import pandas as pd
from langdetect import detect

from google.colab import drive
drive.mount('/content/drive')

# Check for the availability of required packages and install them if necessary
required_packages = ['tensorflow', 'transformers', 'pandas', 'sklearn', 'keras', 'torch', 'langdetect']
for package in required_packages:
    try:
        importlib.import_module(package)
    except ImportError:
        !pip install {package}
        importlib.import_module(package)

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/news_200.csv')

# Create a new DataFrame to store the filtered lines
filtered_df = pd.DataFrame(columns=df.columns)
df = df.dropna(subset=['text'])

df = df.dropna(subset=['title'])
"""
# Iterate through the original DataFrame
for i in range(len(df)):
    try:
        line = str(df.iloc[i]['title'])  # Replace 'title' with the actual column name containing the title
        language = detect(line)

        if language == 'en':
            # Append the line to the filtered DataFrame if the language is English
            filtered_df = filtered_df.append(df.iloc[i], ignore_index=True)
            #print(f"Line {i+1}: Language - {language} (Included)")
        else:
            #print(f"Line {i+1}: Language - {language} (Excluded)")
            pass
    except TypeError:
        print(f"Line {i+1}: Skipped - Non-string text")
        continue

# Save the filtered DataFrame to a new file
filtered_df.to_csv('/content/drive/MyDrive/Colab Notebooks/news_filtered.csv', index=False)
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/news_filtered.csv')
"""
# Tokenization & Formatting
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', do_lower_case=True)

def tokenize_text(df, max_len):
    return tokenizer.batch_encode_plus(
        df['text'].tolist(),
        max_length = max_len,
        pad_to_max_length=True,
        truncation=True,
        return_token_type_ids=False
    )


# Tokenize the text
max_len = 256  # choose a max length
tokens = tokenize_text(df, max_len)

# Model Creation
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)
model = model.to(device)

# Split the dataset into training and testing
train_inputs, test_inputs, train_labels, test_labels = train_test_split(tokens['input_ids'], df['label'].values, random_state=100, test_size=0.2)

# Convert into torch tensors
train_inputs = torch.tensor(train_inputs)
test_inputs = torch.tensor(test_inputs)
train_labels = torch.tensor(train_labels)
test_labels = torch.tensor(test_labels)

# Create DataLoader for the training set
train_data = TensorDataset(train_inputs, train_labels)
train_dataloader = DataLoader(train_data, batch_size=32)

# Training
optimizer = AdamW(model.parameters(), lr=1e-5)

for epoch in range(1):
    model.train()
    for step, batch in enumerate(train_dataloader):
        b_input_ids = batch[0].to(device)
        b_labels = batch[1].to(device)

        outputs = model(b_input_ids, labels=b_labels)

        loss = outputs[0]
        loss.backward()

        optimizer.step()

# Create DataLoader for the test set
test_data = TensorDataset(test_inputs, test_labels)
test_dataloader = DataLoader(test_data, batch_size=32)

# Evaluation
model.eval()

predictions = []
true_labels = []

for batch in test_dataloader:
    batch = [b.to(device) for b in batch]
    b_input_ids, b_labels = batch

    with torch.no_grad():
        outputs = model(b_input_ids)

    # Store predictions and true labels
    logits = outputs.logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    # Get the predicted labels: the output of the model is a probability distribution over the labels
    pred_labels = np.argmax(logits, axis=1)

    predictions.append(pred_labels)
    true_labels.append(label_ids)

# Flatten the predictions and true values for aggregate evaluation on the whole dataset
predictions = np.concatenate(predictions, axis=0)
true_labels = np.concatenate(true_labels, axis=0)

# Print the classification report
print(classification_report(true_labels, predictions, target_names=['Fake News', 'Real News']))


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993224 sha256=26b1ac2e9b24e8bbd68cc578ed9feccd8e1362ad80e16cea373fca20a24be6ba
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect
Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias']
You should pr

              precision    recall  f1-score   support

   Fake News       0.00      0.00      0.00        19
   Real News       0.51      1.00      0.68        20

    accuracy                           0.51        39
   macro avg       0.26      0.50      0.34        39
weighted avg       0.26      0.51      0.35        39



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
import importlib

import numpy as np

package = "tensorflow"
try:
    importlib.import_module(package)
except ImportError:
    !pip install - U {package}
    importlib.import_module(package)
package = "transformers"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "pandas"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "sklearn"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "keras"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "torch"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "transformers"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "langdetect"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

import torch
from sklearn.model_selection import train_test_split
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import classification_report
import pandas as pd
from langdetect import detect

from google.colab import drive
drive.mount('/content/drive')

# Check for the availability of required packages and install them if necessary
required_packages = ['tensorflow', 'transformers', 'pandas', 'sklearn', 'keras', 'torch', 'langdetect']
for package in required_packages:
    try:
        importlib.import_module(package)
    except ImportError:
        !pip install {package}
        importlib.import_module(package)

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/news_500.csv')

# Create a new DataFrame to store the filtered lines
filtered_df = pd.DataFrame(columns=df.columns)
df = df.dropna(subset=['text'])

df = df.dropna(subset=['title'])
"""
# Iterate through the original DataFrame
for i in range(len(df)):
    try:
        line = str(df.iloc[i]['title'])  # Replace 'title' with the actual column name containing the title
        language = detect(line)

        if language == 'en':
            # Append the line to the filtered DataFrame if the language is English
            filtered_df = filtered_df.append(df.iloc[i], ignore_index=True)
            #print(f"Line {i+1}: Language - {language} (Included)")
        else:
            #print(f"Line {i+1}: Language - {language} (Excluded)")
            pass
    except TypeError:
        print(f"Line {i+1}: Skipped - Non-string text")
        continue

# Save the filtered DataFrame to a new file
filtered_df.to_csv('/content/drive/MyDrive/Colab Notebooks/news_filtered.csv', index=False)
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/news_filtered.csv')
"""
# Tokenization & Formatting
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', do_lower_case=True)

def tokenize_text(df, max_len):
    return tokenizer.batch_encode_plus(
        df['text'].tolist(),
        max_length = max_len,
        pad_to_max_length=True,
        truncation=True,
        return_token_type_ids=False
    )


# Tokenize the text
max_len = 256  # choose a max length
tokens = tokenize_text(df, max_len)

# Model Creation
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)
model = model.to(device)

# Split the dataset into training and testing
train_inputs, test_inputs, train_labels, test_labels = train_test_split(tokens['input_ids'], df['label'].values, random_state=100, test_size=0.2)

# Convert into torch tensors
train_inputs = torch.tensor(train_inputs)
test_inputs = torch.tensor(test_inputs)
train_labels = torch.tensor(train_labels)
test_labels = torch.tensor(test_labels)

# Create DataLoader for the training set
train_data = TensorDataset(train_inputs, train_labels)
train_dataloader = DataLoader(train_data, batch_size=32)

# Training
optimizer = AdamW(model.parameters(), lr=1e-5)

for epoch in range(1):
    model.train()
    for step, batch in enumerate(train_dataloader):
        b_input_ids = batch[0].to(device)
        b_labels = batch[1].to(device)

        outputs = model(b_input_ids, labels=b_labels)

        loss = outputs[0]
        loss.backward()

        optimizer.step()

# Create DataLoader for the test set
test_data = TensorDataset(test_inputs, test_labels)
test_dataloader = DataLoader(test_data, batch_size=32)

# Evaluation
model.eval()

predictions = []
true_labels = []

for batch in test_dataloader:
    batch = [b.to(device) for b in batch]
    b_input_ids, b_labels = batch

    with torch.no_grad():
        outputs = model(b_input_ids)

    # Store predictions and true labels
    logits = outputs.logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    # Get the predicted labels: the output of the model is a probability distribution over the labels
    pred_labels = np.argmax(logits, axis=1)

    predictions.append(pred_labels)
    true_labels.append(label_ids)

# Flatten the predictions and true values for aggregate evaluation on the whole dataset
predictions = np.concatenate(predictions, axis=0)
true_labels = np.concatenate(true_labels, axis=0)

# Print the classification report
print(classification_report(true_labels, predictions, target_names=['Fake News', 'Real News']))


In [ ]:
import importlib

import numpy as np

package = "tensorflow"
try:
    importlib.import_module(package)
except ImportError:
    !pip install - U {package}
    importlib.import_module(package)
package = "transformers"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "pandas"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "sklearn"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "keras"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "torch"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "transformers"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "langdetect"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

import torch
from sklearn.model_selection import train_test_split
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import classification_report
import pandas as pd
from langdetect import detect

from google.colab import drive
drive.mount('/content/drive')

# Check for the availability of required packages and install them if necessary
required_packages = ['tensorflow', 'transformers', 'pandas', 'sklearn', 'keras', 'torch', 'langdetect']
for package in required_packages:
    try:
        importlib.import_module(package)
    except ImportError:
        !pip install {package}
        importlib.import_module(package)

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/news_10000.csv')

# Create a new DataFrame to store the filtered lines
filtered_df = pd.DataFrame(columns=df.columns)
df = df.dropna(subset=['text'])

df = df.dropna(subset=['title'])
"""
# Iterate through the original DataFrame
for i in range(len(df)):
    try:
        line = str(df.iloc[i]['title'])  # Replace 'title' with the actual column name containing the title
        language = detect(line)

        if language == 'en':
            # Append the line to the filtered DataFrame if the language is English
            filtered_df = filtered_df.append(df.iloc[i], ignore_index=True)
            #print(f"Line {i+1}: Language - {language} (Included)")
        else:
            #print(f"Line {i+1}: Language - {language} (Excluded)")
            pass
    except TypeError:
        print(f"Line {i+1}: Skipped - Non-string text")
        continue

# Save the filtered DataFrame to a new file
filtered_df.to_csv('/content/drive/MyDrive/Colab Notebooks/news_filtered.csv', index=False)
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/news_filtered.csv')
"""
# Tokenization & Formatting
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', do_lower_case=True)

def tokenize_text(df, max_len):
    return tokenizer.batch_encode_plus(
        df['text'].tolist(),
        max_length = max_len,
        pad_to_max_length=True,
        truncation=True,
        return_token_type_ids=False
    )


# Tokenize the text
max_len = 256  # choose a max length
tokens = tokenize_text(df, max_len)

# Model Creation
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)
model = model.to(device)

# Split the dataset into training and testing
train_inputs, test_inputs, train_labels, test_labels = train_test_split(tokens['input_ids'], df['label'].values, random_state=100, test_size=0.2)

# Convert into torch tensors
train_inputs = torch.tensor(train_inputs)
test_inputs = torch.tensor(test_inputs)
train_labels = torch.tensor(train_labels)
test_labels = torch.tensor(test_labels)

# Create DataLoader for the training set
train_data = TensorDataset(train_inputs, train_labels)
train_dataloader = DataLoader(train_data, batch_size=32)

# Training
optimizer = AdamW(model.parameters(), lr=1e-5)

for epoch in range(1):
    model.train()
    for step, batch in enumerate(train_dataloader):
        b_input_ids = batch[0].to(device)
        b_labels = batch[1].to(device)

        outputs = model(b_input_ids, labels=b_labels)

        loss = outputs[0]
        loss.backward()

        optimizer.step()

# Create DataLoader for the test set
test_data = TensorDataset(test_inputs, test_labels)
test_dataloader = DataLoader(test_data, batch_size=32)

# Evaluation
model.eval()

predictions = []
true_labels = []

for batch in test_dataloader:
    batch = [b.to(device) for b in batch]
    b_input_ids, b_labels = batch

    with torch.no_grad():
        outputs = model(b_input_ids)

    # Store predictions and true labels
    logits = outputs.logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    # Get the predicted labels: the output of the model is a probability distribution over the labels
    pred_labels = np.argmax(logits, axis=1)

    predictions.append(pred_labels)
    true_labels.append(label_ids)

# Flatten the predictions and true values for aggregate evaluation on the whole dataset
predictions = np.concatenate(predictions, axis=0)
true_labels = np.concatenate(true_labels, axis=0)

# Print the classification report
print(classification_report(true_labels, predictions, target_names=['Fake News', 'Real News']))


In [ ]:
import importlib

import numpy as np

package = "tensorflow"
try:
    importlib.import_module(package)
except ImportError:
    !pip install - U {package}
    importlib.import_module(package)
package = "transformers"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "pandas"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "sklearn"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "keras"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "torch"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "transformers"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

package = "langdetect"
try:
    importlib.import_module(package)
except ImportError:
    !pip install {package}
    importlib.import_module(package)

import torch
from sklearn.model_selection import train_test_split
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import classification_report
import pandas as pd
from langdetect import detect

from google.colab import drive
drive.mount('/content/drive')

# Check for the availability of required packages and install them if necessary
required_packages = ['tensorflow', 'transformers', 'pandas', 'sklearn', 'keras', 'torch', 'langdetect']
for package in required_packages:
    try:
        importlib.import_module(package)
    except ImportError:
        !pip install {package}
        importlib.import_module(package)

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv')

# Create a new DataFrame to store the filtered lines
filtered_df = pd.DataFrame(columns=df.columns)
df = df.dropna(subset=['text'])

df = df.dropna(subset=['title'])
"""
# Iterate through the original DataFrame
for i in range(len(df)):
    try:
        line = str(df.iloc[i]['title'])  # Replace 'title' with the actual column name containing the title
        language = detect(line)

        if language == 'en':
            # Append the line to the filtered DataFrame if the language is English
            filtered_df = filtered_df.append(df.iloc[i], ignore_index=True)
            #print(f"Line {i+1}: Language - {language} (Included)")
        else:
            #print(f"Line {i+1}: Language - {language} (Excluded)")
            pass
    except TypeError:
        print(f"Line {i+1}: Skipped - Non-string text")
        continue

# Save the filtered DataFrame to a new file
filtered_df.to_csv('/content/drive/MyDrive/Colab Notebooks/news_filtered.csv', index=False)
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/news_filtered.csv')
"""
# Tokenization & Formatting
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', do_lower_case=True)

def tokenize_text(df, max_len):
    return tokenizer.batch_encode_plus(
        df['text'].tolist(),
        max_length = max_len,
        pad_to_max_length=True,
        truncation=True,
        return_token_type_ids=False
    )


# Tokenize the text
max_len = 256  # choose a max length
tokens = tokenize_text(df, max_len)

# Model Creation
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)
model = model.to(device)

# Split the dataset into training and testing
train_inputs, test_inputs, train_labels, test_labels = train_test_split(tokens['input_ids'], df['label'].values, random_state=100, test_size=0.2)

# Convert into torch tensors
train_inputs = torch.tensor(train_inputs)
test_inputs = torch.tensor(test_inputs)
train_labels = torch.tensor(train_labels)
test_labels = torch.tensor(test_labels)

# Create DataLoader for the training set
train_data = TensorDataset(train_inputs, train_labels)
train_dataloader = DataLoader(train_data, batch_size=32)

# Training
optimizer = AdamW(model.parameters(), lr=1e-5)

for epoch in range(1):
    model.train()
    for step, batch in enumerate(train_dataloader):
        b_input_ids = batch[0].to(device)
        b_labels = batch[1].to(device)

        outputs = model(b_input_ids, labels=b_labels)

        loss = outputs[0]
        loss.backward()

        optimizer.step()

# Create DataLoader for the test set
test_data = TensorDataset(test_inputs, test_labels)
test_dataloader = DataLoader(test_data, batch_size=32)

# Evaluation
model.eval()

predictions = []
true_labels = []

for batch in test_dataloader:
    batch = [b.to(device) for b in batch]
    b_input_ids, b_labels = batch

    with torch.no_grad():
        outputs = model(b_input_ids)

    # Store predictions and true labels
    logits = outputs.logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    # Get the predicted labels: the output of the model is a probability distribution over the labels
    pred_labels = np.argmax(logits, axis=1)

    predictions.append(pred_labels)
    true_labels.append(label_ids)

# Flatten the predictions and true values for aggregate evaluation on the whole dataset
predictions = np.concatenate(predictions, axis=0)
print (predictions)
true_labels = np.concatenate(true_labels, axis=0)

# Print the classification report
print(classification_report(true_labels, predictions, target_names=['Fake News', 'Real News']))


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 76.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 20.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993224 sha256=fae5a64c430cbc3c597248cd09c51533e15060a125026a299beb533a2eb4fb38
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect
Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias']
You should pr

KeyboardInterrupt: ignored